In [2]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import nsforest as ns
from nsforest import utils
from nsforest import preprocessing as pp
from nsforest import nsforesting
from nsforest import evaluating as ev
from nsforest import plotting as pl

In [11]:
adata_ = sc.read_h5ad("/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/01.data/02.atlas_final/2.samap/4.final/Mmus.wb.iter_cluster_annotated.h5ad")

In [12]:
adata_.X = adata_.raw.X
sc.pp.normalize_total(adata_, target_sum=1e4)
sc.pp.log1p(adata_)

In [13]:
cluster_header = "Refined family"
output_folder = "/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/01.data/03.markers/3.nsforest/Mmus/"
label = "Mmus.TFs"
TF = "/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/01.data/07.species_signals/6.TF_vs_species_signals/TFs/Mmus.predicted_TFs.txt"

In [14]:
TF = pd.read_csv(TF, sep = '\t')
TF = TF.loc[TF['prediction'] == True, 'sequence_ID'].tolist()
#adata = adata_[:, adata_.var_names.isin(TF)].copy()
adata = adata[~adata.obs[cluster_header].isin(["Mixture"])].copy()

KeyError: 'Refined family'

In [ ]:
adata.obs.rename(columns={cluster_header: cluster_header.replace(' ','_')}, inplace=True)
cluster_header = cluster_header.replace(' ','_')

In [8]:
ns.pp.dendrogram(adata, cluster_header, save = True, output_folder = output_folder,
                         outputfilename_suffix = label, plot = False)

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
adata = ns.pp.prep_medians(adata, cluster_header, use_mean = True)
adata = ns.pp.prep_binary_scores(adata, cluster_header)
# binary score stored in adata.varm['binary_scores_cell_type']

# plot exp median across cells
plt.clf()
filename = f'{output_folder}/{label}.{cluster_header}.exp_median.png'
print(f"Saving median distributions as...\n{filename}")
a = plt.figure(figsize = (6, 4))
a = plt.hist(adata.varm["medians_" + cluster_header].unstack(), bins = 100)
a = plt.title(f'{"medians_" + cluster_header} histogram')
a = plt.xlabel("medians_" + cluster_header)
a = plt.yscale("log")
a = plt.savefig(filename, bbox_inches='tight')
# plot binary score distribution across cells
plt.clf()
filename = f'{output_folder}/{label}.{cluster_header}.binary_score_distribution.png'
print(f"Saving binary_score distributions as...\n{filename}")
a = plt.figure(figsize = (6, 4))
a = plt.hist(adata.varm["binary_scores_" + cluster_header].unstack(), bins = 100)
a = plt.title(f'{"binary_scores_" + cluster_header} histogram')
a = plt.xlabel("binary_scores_" + cluster_header)
a = plt.yscale("log")
a = plt.savefig(filename, bbox_inches='tight')

In [8]:
results = nsforesting.NSForest(adata, cluster_header, save_supplementary = True, output_folder = output_folder,
        outputfilename_prefix = f'{label}.{cluster_header}', gene_selection = "BinaryFirst_high",
        n_top_genes = 20, n_binary_genes = 20, n_trees = 1500)

to_plot = results.copy()
endrogram = [] # custom dendrogram order
dendrogram = list(adata.uns["dendrogram_" + cluster_header]["categories_ordered"])
to_plot["clusterName"] = to_plot["clusterName"].astype("category")
to_plot["clusterName"] = to_plot["clusterName"].cat.set_categories(dendrogram)
to_plot = to_plot.sort_values("clusterName")
to_plot = to_plot.rename(columns = {"NSForest_markers": "markers"})
to_plot.to_csv(f'{output_folder}/{label}.{cluster_header}.to_plot_results.txt', sep='\t', index=False)

markers_dict = dict(zip(to_plot["clusterName"], to_plot["markers"]))
ns.pl.dotplot(adata, markers_dict, cluster_header, dendrogram = dendrogram, save = True, output_folder = output_folder, outputfilename_suffix = f'{label}.{cluster_header}')
ns.pl.stackedviolin(adata, markers_dict, cluster_header, dendrogram = dendrogram, save = True, output_folder = output_folder, outputfilename_suffix = f'{label}.{cluster_header}')

Preparing adata...
--- 12.967713594436646 seconds ---


KeyError: 'medians_Refined family'

,clusterName,clusterSize,f_score,PPV,recall,TN,FP,FN,TP,marker_count,NSForest_markers,binary_genes,onTarget
0,Astrocytes,6687,0.802252,0.946353,0.498579,52762,189,3353,3334,2,"[MSTRG.20425, MSTRG.6897]","[MSTRG.20425, MSTRG.12684, MSTRG.12777, MSTRG....",0.697384
1,Choroid plexus epithelium,662,0.952381,0.996283,0.809668,58974,2,126,536,2,"[MSTRG.15310, MSTRG.19938]","[MSTRG.15310, MSTRG.584, MSTRG.19938, MSTRG.21...",0.933645
2,Ependymal,5139,0.804309,0.873781,0.610235,54046,453,2003,3136,2,"[MSTRG.14359, MSTRG.6189]","[MSTRG.14862, MSTRG.4943, MSTRG.2341, MSTRG.18...",0.326456
3,Erythrocytes,4955,0.885076,0.961561,0.671443,54550,133,1628,3327,2,"[MSTRG.4, MSTRG.942]","[MSTRG.4, MSTRG.942, MSTRG.19754, MSTRG.4600, ...",0.755854
4,Excitatory dien-/mesencephalon,1458,0.413150,0.488830,0.255144,57791,389,1086,372,3,"[MSTRG.17857, MSTRG.3095, MSTRG.4103]","[MSTRG.17857, MSTRG.3095, MSTRG.4103, MSTRG.64...",0.243430
5,Excitatory telencephalon,3018,0.712448,0.904355,0.385355,56497,123,1855,1163,2,"[MSTRG.14048, MSTRG.11404]","[MSTRG.14048, MSTRG.19795, MSTRG.12881, MSTRG....",0.354587
6,Habenular neurons,2579,0.889443,0.939801,0.732454,56938,121,690,1889,1,[MSTRG.16433],"[MSTRG.8398, MSTRG.16433, MSTRG.6154, MSTRG.77...",0.899223
7,Immune,3723,0.946765,0.972536,0.856030,55825,90,536,3187,2,"[MSTRG.5777, MSTRG.4082]","[MSTRG.5777, MSTRG.7252, MSTRG.4082, MSTRG.767...",0.883996
8,Inhibitory mesencephalon,3873,0.647771,0.841935,0.336948,55520,245,2568,1305,2,"[MSTRG.4456, MSTRG.1033]","[MSTRG.5539, MSTRG.5443, MSTRG.5222, MSTRG.445...",0.333822
9,Inhibitory telencephalon,3249,0.772307,0.982558,0.416128,56365,24,1897,1352,2,"[MSTRG.17344, MSTRG.18138]","[MSTRG.17344, MSTRG.18138, MSTRG.6108, MSTRG.6...",0.885988
